# Etapa 1: o espectro das trilhas de áudio

In [27]:
from configuration import *
from scipy.sparse import csr_matrix
import numpy as np
import pandas as pd
import pickle
import time

%matplotlib inline

In [28]:
print("Usaremos {} amostras.".format(n_samples))

Usaremos 10000 amostras.


In [29]:
df = pd.read_csv(DETECTIONS, delimiter="|", nrows=n_samples)

In [30]:
sources = np.sort(np.array(df['audio_source_id'].drop_duplicates()))

In [31]:
n_features = len(sources)
print("Há {} rádios, dimensões ou features.".format(n_features))

Há 1994 rádios, dimensões ou features.


### Reindexação das _features_ (rádios)

In [32]:
reindexing_map = dict(zip(sources,range(n_features)))

In [33]:
def reindex(source_id):
    return reindexing_map[source_id]

df['source_id'] = df['audio_source_id'].apply(reindex)

### Espectros das trilhas de áudio

In [34]:
def construct_track_spectrum(track_id):
    spectrum = np.zeros(shape=(n_features))
    track = df[df['track_id'] == track_id].groupby(['source_id']).agg(['count'])['audio_source_id']
    absolute_freq = track.sum()
    track = track/absolute_freq
    for idx, value in track.itertuples():
        spectrum[idx] = value
    return spectrum

In [35]:
track_ids = np.sort(np.array(df['track_id'].drop_duplicates()))

In [36]:
print("Há {} músicas.".format(len(track_ids)))

Há 5151 músicas.


In [37]:
start = time.time()

tracks = np.zeros(shape=(len(track_ids),n_features), dtype=np.float64)
i = 0
for track_id in track_ids:
    tracks[i,:] = construct_track_spectrum(track_id)
    i += 1

end = time.time()
print("Levou {0:.3} s para construir todos os espectros.".format(end-start))

Levou 29.8 s para construir todos os espectros.


###  Persistência dos espectros

Persiste os espectros num formato compactado e apropriado para o algoritmo de classificação:

In [38]:
assert not np.any(np.isnan(tracks))
assert np.max(tracks) <= 1.0
assert np.min(tracks) >= 0.0

In [39]:
sparse_matrix = csr_matrix(tracks)
np.savez(TRACKS,
         data = sparse_matrix.data,
         indices = sparse_matrix.indices,
         indptr = sparse_matrix.indptr,
         shape = sparse_matrix.shape)

### Persistência dos track_ids

A lista de track_ids ordenada será necessária no próximo estágio para associar cada track_id com um ID de gênero musical. Para não ter de criar novamente essa lista, salvamo-la em disco:

In [40]:
pickle.dump(track_ids, file(TRACK_IDS, 'wb'), pickle.HIGHEST_PROTOCOL)